实现乘方库函数```pow()```的整数运算。

In [23]:
# 需要考虑底数为0，指数为0，指数为负数这三种情况
# 当指数为偶数时，可以使用递归求解
# 当指数为奇数时，还要单独乘一次基数
def power(base, exp):
    # 几个基准值
    if base == 0:
        return 0
    if exp == 0:
        return 1
    if exp == 1:
        return base

    neg_flag = 1 if exp < 0 else 0  # 负指数标记
    exp = abs(exp)

    num = power(base, exp >> 1)
    num *= num
    if exp & 0x01 == 1:  # 判断是否奇数
        num *= base

    if neg_flag:
        num = 1 / num

    return num


power(-2, -2)

0.25

实现"."与"*"的正则匹配，前者代表任意字符，后者代表任意次数。